<a href="https://colab.research.google.com/github/Gaurav-phatkare/arXiv-Paper-Abstracts-Mutli-Label-Classification-Using-Deep-Learning/blob/main/Multilabel_Text_Classification_of_Research_Papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!nvidia-smi

Fri Aug  4 05:49:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [10]:
!pip install kaggle

In [11]:
!mkdir ~/.kaggle

In [12]:
!cp kaggle.json ~/.kaggle/

In [13]:
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d spsayakpaul/arxiv-paper-abstracts

 99% 44.0M/44.6M [00:03<00:00, 16.3MB/s]
100% 44.6M/44.6M [00:03<00:00, 13.0MB/s]


In [15]:
!unzip /content/arxiv-paper-abstracts.zip

Archive:  /content/arxiv-paper-abstracts.zip
  inflating: arxiv_data.csv          
  inflating: arxiv_data_210930-054931.csv  


In [16]:
df = pd.read_csv("/content/arxiv_data.csv")

In [17]:
df.shape

(51774, 3)

In [18]:
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [19]:
df.iloc[4].summaries,df.iloc[4].terms

('To ensure safety in automated driving, the correct perception of the\nsituation inside the car is as important as its environment. Thus, seat\noccupancy detection and classification of detected instances play an important\nrole in interior sensing. By the knowledge of the seat occupancy status, it is\npossible to, e.g., automate the airbag deployment control. Furthermore, the\npresence of a driver, which is necessary for partially automated driving cars\nat the automation levels two to four can be verified. In this work, we compare\ndifferent statistical methods from the field of image segmentation to approach\nthe problem of background-foreground segmentation in camera based interior\nsensing. In the recent years, several methods based on different techniques\nhave been developed and applied to images or videos from different\napplications. The peculiarity of the given scenarios of interior sensing is,\nthat the foreground instances and the background both contain static as well as\

In [20]:
df['titles'].duplicated().sum()

12802

In [21]:
df.drop_duplicates(inplace = True)

In [22]:
df.shape

(38991, 3)

In [23]:
import re
def Text_cleaning(text):
  text = re.sub(r"\n" , " ", text)

  text = re.sub(r"\s+" , " ", text)

  text = text.lower()
  text = text.replace('\n'," ")

  return text



In [24]:
Text_cleaning(df.iloc[4].summaries)

'to ensure safety in automated driving, the correct perception of the situation inside the car is as important as its environment. thus, seat occupancy detection and classification of detected instances play an important role in interior sensing. by the knowledge of the seat occupancy status, it is possible to, e.g., automate the airbag deployment control. furthermore, the presence of a driver, which is necessary for partially automated driving cars at the automation levels two to four can be verified. in this work, we compare different statistical methods from the field of image segmentation to approach the problem of background-foreground segmentation in camera based interior sensing. in the recent years, several methods based on different techniques have been developed and applied to images or videos from different applications. the peculiarity of the given scenarios of interior sensing is, that the foreground instances and the background both contain static as well as dynamic eleme

In [25]:
df['cleaned_summary'] = df['summaries'].apply(Text_cleaning)

In [26]:
df.head()

,titles,summaries,terms,cleaned_summary
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']",stereo matching is one of the widely used tech...
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']",the recent advancements in artificial intellig...
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']","in this paper, we proposed a novel mutual cons..."
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV'],consistency training has proven to be an advan...
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']","to ensure safety in automated driving, the cor..."


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38991 entries, 0 to 51772
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titles           38991 non-null  object
 1   summaries        38991 non-null  object
 2   terms            38991 non-null  object
 3   cleaned_summary  38991 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [28]:
sum(df['terms'].value_counts() == 1)

2321

In [29]:
new_df = df.groupby('terms').filter(lambda x:len(x) > 1)
new_df.shape

(36670, 4)

In [30]:
new_df.duplicated().sum()

0

## convert the string labels to lists of strings

In [31]:
import ast
new_df['terms'] = new_df['terms'].apply(lambda x: ast.literal_eval(x))

In [32]:
new_df.terms.value_counts()

[cs.CV]                                               12753
[cs.LG, stat.ML]                                       4076
[cs.LG]                                                2047
[cs.CV, cs.LG]                                         1488
[cs.LG, cs.AI]                                         1207
                                                      ...  
[cs.CV, cs.LG, eess.IV, eess.SP, stat.ML]                 2
[cs.LG, cs.CL, cs.CV, cs.NE, stat.ML]                     2
[stat.ML, cs.IT, cs.LG, math.IT, math.ST, stat.TH]        2
[cs.CV, cs.CL, cs.LG, eess.AS, eess.IV]                   2
[cs.LG, G.3]                                              2
Name: terms, Length: 836, dtype: int64

## Spliting into train test split

In [33]:
from sklearn.model_selection import train_test_split
# train_df, test_df_0 = train_test_split(
#     new_df,
#     test_size = 0.1,
#     stratify = new_df['terms'].values
# )

In [34]:
# train_df.shape, test_df_0.shape

In [35]:
# # spliting test data further into test and validation

# test_df, val_df = train_test_split(
#     test_df_0,
#     train_size = 0.5,
#     stratify = new_df['terms'].values
# )

In [36]:
from sklearn.model_selection import train_test_split

def split_dataset(data, train_ratio=0.8, test_ratio=0.1, val_ratio=0.1, random_seed=42):

    train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=random_seed)
    test_data, val_data = train_test_split(temp_data, train_size=test_ratio / (test_ratio + val_ratio),
                                           random_state=random_seed)

    return train_data, test_data, val_data


train_data, test_data, val_data = split_dataset(new_df, train_ratio=0.8, test_ratio=0.1, val_ratio=0.1)


In [37]:
len(train_data),len(test_data),len(val_data)

(29336, 3667, 3667)

### Multi-label Binarization

In [38]:
terms = tf.ragged.constant(train_data['terms'].values)
lookup = tf.keras.layers.StringLookup(output_mode = 'multi_hot')
lookup.adapt(terms)
vocab = lookup.get_vocabulary()

In [41]:
vocab

['[UNK]',
 'cs.CV',
 'cs.LG',
 'stat.ML',
 'cs.AI',
 'eess.IV',
 'cs.RO',
 'cs.CL',
 'cs.NE',
 'cs.CR',
 'math.OC',
 'eess.SP',
 'cs.GR',
 'cs.SI',
 'cs.MM',
 'cs.SY',
 'cs.IR',
 'eess.SY',
 'cs.MA',
 'cs.HC',
 'cs.DC',
 'math.IT',
 'cs.IT',
 'cs.CY',
 'stat.AP',
 'stat.TH',
 'math.ST',
 'stat.ME',
 'eess.AS',
 'cs.SD',
 'q-bio.QM',
 'cs.DS',
 'q-bio.NC',
 'cs.NI',
 'cs.SE',
 'I.2.6',
 'stat.CO',
 'cs.CG',
 'cs.GT',
 'math.NA',
 'physics.chem-ph',
 'cs.NA',
 'cs.DB',
 'cs.LO',
 'cs.PL',
 '68T45',
 'q-bio.BM',
 'physics.comp-ph',
 'math.PR',
 'cs.AR',
 'cond-mat.dis-nn',
 'math.DS',
 'I.2.10',
 'cs.CE',
 'physics.data-an',
 'quant-ph',
 '68T05',
 'q-fin.ST',
 'cond-mat.stat-mech',
 'cs.DM',
 'physics.soc-ph',
 'cs.CC',
 'astro-ph.IM',
 'I.4.6',
 'q-bio.GN',
 'econ.EM',
 'physics.med-ph',
 'cs.PF',
 'I.4.8',
 'q-fin.TR',
 'physics.ao-ph',
 '68U10',
 'math.AT',
 'I.2',
 'hep-ex',
 'cs.FL',
 'cond-mat.mtrl-sci',
 'I.5.4',
 'I.4',
 'physics.optics',
 'physics.geo-ph',
 'math.AP',
 'I.4; I.5

In [42]:
def invert_multi_hot(encoded_labels):
  hot_indices = np.argwhere(encoded_labels == 1.0)[...,0]
  return np.take(vocab, hot_indices)

In [43]:
import numpy as np
ind = np.argwhere([1,0,0,1,0])[...,0]
np.take(['a','b','c','d','e'],ind)

array(['a', 'd'], dtype='<U1')

**Try multi hot with some sample**

In [47]:
sample_label = train_data['terms'].iloc[0]
sample_label

['cs.CV', 'cs.LG', 'eess.IV']

In [49]:
label_binarized = lookup(sample_label)
label_binarized

<tf.Tensor: shape=(151,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

**Invert binarized labels to sample labels**

In [50]:
invert_multi_hot(label_binarized)

array(['cs.CV', 'cs.LG', 'eess.IV'], dtype='<U40')

In [53]:
def length(x):
  x = x.split(" ")
  return len(x)

In [54]:
train_data['cleaned_summary'].apply(length).describe()

count    29336.000000
mean       171.597491
std         45.364715
min          5.000000
25%        140.000000
50%        169.000000
75%        201.000000
max        498.000000
Name: cleaned_summary, dtype: float64

## Preprocessing dataset

In [55]:
max_len = 160
batch_size = 32
padding_token = '<pad>'
auto = tf.data.AUTOTUNE


def make_dataset(dataframe, is_train=True):
  labels = tf.ragged.constant(dataframe['terms'].values)
  label_binarized = lookup(labels).numpy()
  dataset = tf.data.Dataset.from_tensor_slices(
      (dataframe['cleaned_summary'].values, label_binarized)
  )
  dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
  return dataset.batch(batch_size)

In [56]:
train_dataset = make_dataset(train_data,is_train=True)
val_dataset = make_dataset(val_data,is_train=False)
test_dataset = make_dataset(test_data,is_train=False)

In [57]:
text_batch, label_batch = next(iter(train_dataset))

for i, text in enumerate(text_batch[:5]):
  label = label_batch[i].numpy()[None,...]
  print(f'Abstract:{text}')
  print(f"labels: {invert_multi_hot(label[0])}")
  print(" ")

Abstract:b'despite the substantial progress of active learning for image recognition, there still lacks an instance-level active learning method specified for object detection. in this paper, we propose multiple instance active object detection (mi-aod), to select the most informative images for detector training by observing instance-level uncertainty. mi-aod defines an instance uncertainty learning module, which leverages the discrepancy of two adversarial instance classifiers trained on the labeled set to predict instance uncertainty of the unlabeled set. mi-aod treats unlabeled images as instance bags and feature anchors in images as instances, and estimates the image uncertainty by re-weighting instances in a multiple instance learning (mil) fashion. iterative instance uncertainty learning and re-weighting facilitate suppressing noisy instances, toward bridging the gap between instance uncertainty and image-level uncertainty. experiments validate that mi-aod sets a solid baseline 